### Import Libraries

In [58]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import tensorflow as tf
from shared.models import SAT1Start
from shared.generators import SAT1DataGenerator
from shared.utilities import earlyStopping_cb

%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [59]:
data_name = 'data21-25_AC.npz'
data_path = Path('data/sat1') / data_name

with np.load(data_path) as f_data:
    data = f_data['data']
    labels = f_data['labels']
    participants = f_data['participants']
    
data = data.reshape(-1, 30, 210, 1)

In [60]:
categories = sorted(list(set(labels.flatten())))
x_train = data[np.isin(participants, ['0021', '0022', '0023', '0024'])]
y_train = labels[np.isin(participants, ['0021', '0022', '0023', '0024'])]

x_test = data[np.isin(participants, ['0025'])]
y_test = labels[np.isin(participants, ['0025'])]

train_gen = SAT1DataGenerator(x_train, y_train)
test_gen = SAT1DataGenerator(x_test, y_test)

In [70]:
# # Run before re-training to clear up VRAM
import gc
gc.collect()
tf.keras.backend.clear_session()
del model

In [71]:
# Instantiate model for 210 sample length, 30 electrodes/features, and 4 classes
model = SAT1Start(30, 210, 4)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [72]:
# model.fit(train_gen,
#           epochs=20,
#           # TODO: Create val_gen (use 6th participant?)
#           validation_data=test_gen,
#           callbacks=[earlyStopping_cb])
model.fit(train_gen,
          epochs=20,
          # TODO: Create val_gen (use 6th participant?)
          validation_data=test_gen)

Epoch 1/20


2023-05-30 10:19:50.572357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-30 10:19:51.011440: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


60/60 [==============================] - 2s 16ms/step - loss: 1.5290 - accuracy: 0.2729 - val_loss: 1.3763 - val_accuracy: 0.3255
Epoch 2/20
 1/60 [..............................] - ETA: 0s - loss: 1.4822 - accuracy: 0.4375

2023-05-30 10:19:52.463323: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


60/60 [==============================] - 1s 9ms/step - loss: 1.4796 - accuracy: 0.3203 - val_loss: 1.5245 - val_accuracy: 0.3255
Epoch 3/20
60/60 [==============================] - 1s 10ms/step - loss: 1.4488 - accuracy: 0.3161 - val_loss: 1.6495 - val_accuracy: 0.3255
Epoch 4/20
60/60 [==============================] - 1s 9ms/step - loss: 1.3697 - accuracy: 0.3551 - val_loss: 1.7234 - val_accuracy: 0.3255
Epoch 5/20
60/60 [==============================] - 1s 9ms/step - loss: 1.3512 - accuracy: 0.3878 - val_loss: 1.9684 - val_accuracy: 0.3255
Epoch 6/20
60/60 [==============================] - 1s 9ms/step - loss: 1.2814 - accuracy: 0.4162 - val_loss: 1.9287 - val_accuracy: 0.3255
Epoch 7/20
60/60 [==============================] - 1s 9ms/step - loss: 1.2257 - accuracy: 0.4447 - val_loss: 1.6687 - val_accuracy: 0.3373
Epoch 8/20
60/60 [==============================] - 1s 9ms/step - loss: 1.1238 - accuracy: 0.5058 - val_loss: 1.3280 - val_accuracy: 0.4471
Epoch 9/20
60/60 [============